In [ ]:
!pip install -U sentence-transformers

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, losses, evaluation
from sentence_transformers.readers import InputExample
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

In [2]:
df = pd.read_csv('/content/Resume_to_Jobs_Matching.csv')
df.head()

,Resume Description,Job Description,Match
0,"Пол, возраст: Мужчина , 47 лет , родился 1 се...","Название: Менеджер по продажам\n, Цена: от 60 ...",1
1,"Пол, возраст: Мужчина , 27 лет , родился 7 се...","Название: Менеджер по продажам\n, Цена: от 90 ...",1
2,"Пол, возраст: Мужчина , 21 год , родился 20 н...","Название: Водитель\n, Цена: от 60 000 ₽\n, Усл...",1
3,"Пол, возраст: Мужчина , 45 лет , родился 22 д...","Название: Менеджер по продажам\n, Цена: от 90 ...",1
4,"Пол, возраст: Мужчина , 30 лет , родился 28 м...",Header: Java-разработчик в Контест; Emoji: 🏆; ...,0


In [3]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [4]:
train_examples = [InputExample(texts=[e1,e2], label=float(y)) for e1, e2, y in test_df.to_numpy()]
test_examples = [InputExample(texts=[e1,e2], label=float(y)) for e1, e2, y in test_df.to_numpy()]

In [5]:
batch_size = 6
epochs = 5

In [6]:
evaluator=evaluation.BinaryClassificationEvaluator(
        sentences1=test_df['Job Description'].tolist(),
        sentences2=test_df['Resume Description'].tolist(),
        labels=test_df['Match'].tolist(),
        batch_size=batch_size,
        name='bceval',
        show_progress_bar=True,
        # write_csv=True
    )

In [7]:
score = {}
best_score = (0,0)
for lr in tqdm(np.linspace(1e-5, 1e-2, num=3)):
    model = SentenceTransformer('cointegrated/rubert-tiny2')
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
    train_loss = losses.ContrastiveLoss(model=model)
    score[lr] = []
    score[lr].append(evaluator(model))
    for epoch in tqdm(range(epochs)):
        model.fit(
            [(train_dataloader, train_loss)],
            epochs=epochs,
            optimizer_params={'lr': lr},
            optimizer_class=torch.optim.AdamW,
        )
        cur_score = evaluator(model)
        score[lr].append(cur_score)
        if cur_score > best_score[0]:
            best_score = (cur_score, (lr, epoch))
            torch.save(model, 'best_model.pth')
    print(score[lr])


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/51 [00:00<?, ?it/s]


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/31 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/31 [00:00<?, ?it/s]

Iteration:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/51 [00:00<?, ?it/s]


100%|██████████| 3/3 [14:15<00:00, 285.07s/it]

[0.9874540151394575, 1.0, 1.0, 1.0, 1.0, 1.0]


In [8]:
score

{1e-05: [0.9874540151394575,
  0.9885667323001575,
  0.9901649490904896,
  0.9909824863504206,
  0.992013125236156,
  0.9922178381718262],
 0.005005: [0.9874540151394575, 1.0, 1.0, 1.0, 1.0, 1.0],
 0.01: [0.9874540151394575, 1.0, 1.0, 1.0, 1.0, 1.0]}

In [39]:
job_embeds = model.encode(df['Job Description'].tolist())

In [40]:
resume = "Женщина, 25 лет, Java-разработчик"
resume_embeds = model.encode(resume)

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
res = cosine_similarity([resume_embeds],job_embeds)

In [55]:
#Подходящяя позиция
print(df['Job Description'].iloc[np.argmax(res)])

Разработчик фронтенда в Контест🏆

Яндекс.Контест — это платформа для автоматической проверки заданий и проведения олимпиад. Отличительная особенность платформы — на ней можно проверять задачи по программированию. Платформа Контест используется не только в Яндексе для отбора в ШАД и на стажировки или на ежегодном чемпионате Яндекса по программированию, но и на внешних мероприятия, таких как региональный этап всероссийской олимпиады школьников по информатике и «Я — профессионал». Платформа развивается, и мы ищем новых людей.

Мы ждем, что вы:
• имеете опыт фронтенд-разработки от двух лет
• работали над большими проектами
• знаете, что такое UX и как его использовать для создания удобных и функциональных интерфейсов
• имели дело с современными JS-фреймворками (React, Vue, Angular)
• понимаете устройство Node.js

Что нужно делать:
• участвовать в создании проекта
• обсуждать фичи и участвовать в их реализации
• создавать и улучшать интерфейсы

Будет плюсом, если вы:
• разбираетесь в бэкенд